In [1]:
from coffea.nanoevents import NanoEventsFactory, NanoAODSchema, BaseSchema,PFNanoAODSchema
import awkward as ak
import uproot
from matplotlib import pyplot as plt

# print coffea version
import coffea
print(coffea.__version__)

print(ak.__version__)


0.7.22
1.10.3


In [10]:
filename = "root://xrootd-cms.infn.it//store/mc/Run3Summer22NanoAODv12/QCD_PT-15_TuneCP5_Flat2018_13p6TeV_pythia8/NANOAODSIM/JMENano12p5_132X_mcRun3_2022_realistic_v3-v2/70000/1a3d12b4-15b7-415c-a25a-df0e81f71648.root"
# filename = "root://storage01.lcg.cscs.ch:1096//pnfs/lcg.cscs.ch/cms/trivcat//store/mc/Run3Summer22EENanoAODv12/GluGlutoHHto4B_kl-1p00_kt-1p00_c2-0p00_TuneCP5_13p6TeV_powheg-pythia8/NANOAODSIM/Poisson60KeepRAW_130X_mcRun3_2022_realistic_postEE_v6-v2/50000/fc603037-ef65-4bbf-9cef-934ecec40bbe.root"
# filename = "/pnfs/psi.ch/cms/trivcat/store/user/mmalucch/hh4b_12.root"
print("Opening file:", filename)
# load 100 events
events = NanoEventsFactory.from_root(
    filename, schemaclass=NanoAODSchema, entry_stop=100
).events()
print(len(events))
# open file with uproot
# file = uproot.open(filename)
# # get the tree
# events = file['Events']
# print("Events read:", len(events))
# print(events.keys())
# print(events, type(events))

# convert the tree to a awkward array
# events = events.arrays(library="ak", how="zip")
# print(events, type(events))

Opening file: root://xrootd-cms.infn.it//store/mc/Run3Summer22NanoAODv12/QCD_PT-15_TuneCP5_Flat2018_13p6TeV_pythia8/NANOAODSIM/JMENano12p5_132X_mcRun3_2022_realistic_v3-v2/70000/1a3d12b4-15b7-415c-a25a-df0e81f71648.root
100


In [11]:
neutrinos=events.GenPart[(abs(events.GenPart.pdgId)==12) | (abs(events.GenPart.pdgId)==14) |(abs(events.GenPart.pdgId)==16)]
print(neutrinos)
print(neutrinos.genPartIdxMother)
# print the neutrinos mother
neutrino_mother=events.GenPart[neutrinos.genPartIdxMother].pdgId
print(neutrino_mother)


[[], [], [], [], [], [], [], ... [], [], [], [GenParticle], [GenParticle], []]
[[], [], [], [], [], [], [], [30], [], ... [], [9], [], [], [], [58], [30], []]
[[], [], [], [], [], [], [], [411], [], ... [511], [], [], [], [-511], [-411], []]


In [12]:
# remove empty arrays
pi_neutrino=(neutrinos[(abs(neutrino_mother)>=100) & (abs(neutrino_mother)<=400)])
pi_neutrino_mother=(neutrino_mother[(abs(neutrino_mother)>=100) & (abs(neutrino_mother)<=400)])
print(ak.sum(ak.num(pi_neutrino)))
print(ak.sum(ak.num(pi_neutrino_mother)))
print(pi_neutrino, len(pi_neutrino))
print(pi_neutrino_mother, len(pi_neutrino_mother))

0
0
[[], [], [], [], [], [], [], [], [], [], ... [], [], [], [], [], [], [], [], [], []] 100
[[], [], [], [], [], [], [], [], [], [], ... [], [], [], [], [], [], [], [], [], []] 100


In [13]:
# print only the arrays with at least one element
idx=2
# print(pi_neutrino[ak.num(pi_neutrino)>0])
print("child",neutrinos[ak.num(neutrinos)>0].pdgId)
# print(neutrinos[ak.num(neutrinos)>0].status)
print("mother",neutrino_mother[ak.num(neutrino_mother)>0])
neutrino_mother_mother=events.GenPart[events.GenPart[neutrinos.genPartIdxMother].genPartIdxMother]
print("granma", neutrino_mother_mother[ak.num(neutrino_mother_mother)>0].pdgId[idx])
print("child of granma", neutrino_mother_mother[ak.num(neutrino_mother_mother)>0].children.pdgId[idx])
neutrinomother_mother_mother=events.GenPart[events.GenPart[events.GenPart[neutrinos.genPartIdxMother].genPartIdxMother].genPartIdxMother]
print("great granma", neutrinomother_mother_mother[ak.num(neutrinomother_mother_mother)>0].pdgId[idx])
print("child of great granma", neutrinomother_mother_mother[ak.num(neutrinomother_mother_mother)>0].children.pdgId[idx])
neutrinomother_mother_mother_mother=events.GenPart[events.GenPart[events.GenPart[events.GenPart[neutrinos.genPartIdxMother].genPartIdxMother].genPartIdxMother].genPartIdxMother]
print("great great granma", neutrinomother_mother_mother_mother[ak.num(neutrinomother_mother_mother_mother)>0].pdgId[idx])
print("child of great great granma", neutrinomother_mother_mother_mother[ak.num(neutrinomother_mother_mother_mother)>0].children.pdgId[idx])


child [[12], [-16, 16, -14], [-12], [12], [-12], ... [16, -16], [-12], [14], [-14], [-14]]
mother [[411], [-511, 15, -411], [-411], [411], ... -15], [-421], [511], [-511], [-411]]
granma [-11]
child of granma [[]]
great granma [2]
child of great granma [[2, 21, 21, 11, -11]]
great great granma [2]
child of great great granma [[2]]


In [14]:

leptons=events.GenPart[(abs(events.GenPart.pdgId)==11) | (abs(events.GenPart.pdgId)==13) |(abs(events.GenPart.pdgId)==15)]
print(leptons.pdgId)
lepton_mother=events.GenPart[leptons.genPartIdxMother].pdgId
print(lepton_mother)

non_meson_neutrinos=neutrino_mother
print(len(non_meson_neutrinos[ak.num(non_meson_neutrinos)>0]))
non_meson_neutrinos=neutrino_mother[(abs(neutrino_mother)>400)]
print(len(non_meson_neutrinos[ak.num(non_meson_neutrinos)>0]))
non_meson_neutrinos=neutrino_mother[(abs(neutrino_mother)>500)]
print(len(non_meson_neutrinos[ak.num(non_meson_neutrinos)>0]))
non_meson_neutrinos=neutrino_mother[(abs(neutrino_mother)>1000)]
print("brion",(non_meson_neutrinos[ak.num(non_meson_neutrinos)>0]))

non_meson_neutrinos=neutrinos[(abs(neutrino_mother)<100) & (abs(neutrino_mother)>20)]
print(len(non_meson_neutrinos[ak.num(non_meson_neutrinos)>0].pdgId))
print((neutrino_mother[ak.num(non_meson_neutrinos)>0]))

[[11, -11, 11, -11, 11, -11], [11, -11], ... -11, 13, 11, -11, 11, -11], [11, -11]]
[[2, 2, 1, 1, 1, 1], [111, 111], [2, 2, ... [4, 4, -411, 4, 4, 4, 4], [111, 111]]
23
23
7
brion [[-4122]]
0
[]


In [15]:
n_mother_flatten=ak.flatten(neutrino_mother)
print(n_mother_flatten)
# plt.hist(n_mother_flatten, bins=12000, range=(-6000,6000))

[411, -511, 15, -411, -411, 411, -421, 421, ... 421, 511, -15, -421, 511, -511, -411]


In [31]:
gen_jet=events.GenJet

deltaR_unflatten=gen_jet.metric_table(neutrinos)
print(deltaR_unflatten)
deltaR=ak.flatten(deltaR_unflatten, axis=2)
print(deltaR[3:10])
mask_04=deltaR<0.4
print(deltaR[mask_04][3:10])
num_04=ak.num(deltaR[mask_04])
print(num_04)
print(num_04[num_04>0])
deltaR_04=deltaR[mask_04][num_04>0]
print(deltaR_04)

print(ak.count(deltaR[mask_04], axis=1))

print(deltaR[-2][mask_04[-2]])

[[[], [], [], [], [], []], [[], [], ... [3.04], [8.03], [5.27]], [[], [], [], []]]
[[], [], [], ... 0.136, 0.508, 2.75, 2.99, 0.427, 3.76, 3.02, 2.1, 2.86], [], []]
[[], [], [], [], [0.136], [], []]
[0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, ... 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]
[1, 3, 2, 1, 1, 1, 1, 1, 1, 3, 1, 2, 3]
[[0.136], [0.221, 0.246, 0.281], [0.399, ... [0.0949, 0.38], [0.343, 0.339, 0.352]]
[0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, ... 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]
[]
